# [SOLUTION] Exercise - Create an Agent with external API call enabled

In this exercise, you'll build an agent that can interact with external APIs to gather real-time data
and provide responses based on that information. You'll combine concepts from state management and
memory while adding the ability to make external API calls safely and effectively.


## Challenge

Your task is to create an agent that can make External API Calls:

- Implement tools that interact with real APIs
- Handle API responses and errors gracefully
- Use environment variables for API keys
- Process and format API data for user consumption

## Setup
First, let's import the necessary libraries:

In [2]:
import os
import random
from typing import List
import requests
from dotenv import load_dotenv
import os

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [3]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_api_key

'voc-2015200036168865441126268e0ed5be47456.24871461'

## Define API tools

Feel free to use any open service available through APIs.

Here are a few examples. You can follow the instructions given.
- https://jsonplaceholder.typicode.com/guide/
- https://www.exchangerate-api.com/
- https://openweathermap.org/

Or you can find one you're interested in here.
- https://github.com/public-apis/public-apis

In [4]:
@tool
def get_weather(city: str) -> dict:
    """
    Get current weather for a city using OpenWeather API
    args:
        city (str): Name of the city to get weather for
    """
    API_KEY = os.getenv("OPENWEATHER_API_KEY")
    BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
    
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"
    }
    
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    return response.json()

In [5]:
@tool
def get_exchange_rate(from_currency: str = "USD") -> dict:
    """
    Get latest exchange rates from a base currency
    args:
        from_currency (str): Base currency code (default: USD)
    """
    API_KEY = os.getenv("EXCHANGERATE_API_KEY")
    BASE_URL = "https://v6.exchangerate-api.com/v6"
    
    url = f"{BASE_URL}/{API_KEY}/latest/{from_currency}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

In [6]:
@tool
def get_random_pokemon() -> dict:
    """Get a random Pokemon from the original 151"""
    URL = "https://pokeapi.co/api/v2/pokemon?limit=151"
    response = requests.get(URL)
    response.raise_for_status()
    return random.choice(response.json()['results'])

In [9]:
tools = [get_weather, get_exchange_rate, get_random_pokemon]

In [10]:
agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are an assistant that can help with:\n"
        "1. Getting weather information for cities\n"
        "2. Checking currency exchange rates\n"
        "3. Getting a random Pokemon\n"
        "Use the available tools to help answer questions about these topics.\n"
        "Maintain context across conversations within the same session."
    ),
    tools=tools,
    api_key=openai_api_key
)

In [11]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agent

In [12]:
session_id = "external_tools"

In [13]:
run1 = agent.invoke(
    query="What's the weather like in London?", 
    session_id=session_id,
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep


APIConnectionError: Connection error.

In [ ]:
run2 = agent.invoke(
    query="What's the exchange rate from USD to EUR?", 
    session_id=session_id,
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are an assistant that can help with:
1. Getting weather information for cities
2. Checking currency exchange rates
3. Getting a random Pokemon
Use the available tools to help answer questions about these topics.
Maintain context across conversations within the same session., tool_calls = None)
 -> (role = user, content = What's the weather like in London?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_0Nj195qnSdtyy16iWfsJw7QE', function=Function(arguments='{"city":"London"}', name='get_weather'), type='function')])
 -> (role = tool, content = "{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 800, 'ma

In [ ]:
run3 = agent.invoke(
    query="Pick one random Pokemon!", 
    session_id=session_id,
)
print("\nMessages from run 3:")
messages = run3.get_final_state()["messages"]
print_messages(messages)


[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = You are an assistant that can help with:
1. Getting weather information for cities
2. Checking currency exchange rates
3. Getting a random Pokemon
Use the available tools to help answer questions about these topics.
Maintain context across conversations within the same session., tool_calls = None)
 -> (role = user, content = What's the weather like in London?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_0Nj195qnSdtyy16iWfsJw7QE', function=Function(arguments='{"city":"London"}', name='get_weather'), type='function')])
 -> (role = tool, content = "{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 800, 'ma

## Check session histories

In [ ]:
runs = agent.get_session_runs(session_id)
for i, run_object in enumerate(runs, 1):
    print(f"\n# Run {i}", run_object.metadata)
    print("Messages:")
    print_messages(run_object.get_final_state()["messages"])


# Run 1 {'run_id': '0f742e3c-26df-48c1-a3ea-f1de16066a0b', 'start_timestamp': '2025-07-03 09:09:48.010245', 'end_timestamp': '2025-07-03 09:09:53.680763', 'snapshot_counts': 5}
Messages:
 -> (role = system, content = You are an assistant that can help with:
1. Getting weather information for cities
2. Checking currency exchange rates
3. Getting a random Pokemon
Use the available tools to help answer questions about these topics.
Maintain context across conversations within the same session., tool_calls = None)
 -> (role = user, content = What's the weather like in London?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_0Nj195qnSdtyy16iWfsJw7QE', function=Function(arguments='{"city":"London"}', name='get_weather'), type='function')])
 -> (role = tool, content = "{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'tem

In [ ]:
runs = agent.get_session_runs(session_id)
for run_object in runs:
    print(run_object)
    for snp in run_object.snapshots:
        print(f"-> {snp}")
    print("\n")

Run('0f742e3c-26df-48c1-a3ea-f1de16066a0b')
-> Snapshot('3a00dad3-1bf5-4f1a-b4dd-d94f7c28efa9') @ [2025-07-03 09:09:48.010430]: __entry__.State({'user_query': "What's the weather like in London?", 'instructions': 'You are an assistant that can help with:\n1. Getting weather information for cities\n2. Checking currency exchange rates\n3. Getting a random Pokemon\nUse the available tools to help answer questions about these topics.\nMaintain context across conversations within the same session.', 'messages': [], 'current_tool_calls': None, 'session_id': 'external_tools'})
-> Snapshot('74868b1d-9822-463b-8c78-a19886ef629f') @ [2025-07-03 09:09:48.010626]: message_prep.State({'user_query': "What's the weather like in London?", 'instructions': 'You are an assistant that can help with:\n1. Getting weather information for cities\n2. Checking currency exchange rates\n3. Getting a random Pokemon\nUse the available tools to help answer questions about these topics.\nMaintain context across conve